# Домашнее задание № 1

## Задание 1 (2 балла)

Найдите 1 любой способ сломать токенизацию на предложения функцией sentenize из библиотеки razdel. Придумайте (или найдите на каком-то корпусе) такое предложение (или несколько предложений), которое будет некорректно разобрано sentenize, но при этом будет грамматически корректным. 

Из Лурка (старого — у меня локальный дамп)

> «Кто хочет стать миллионером?» — педерача, в которой актеры гонятся за 3кк рублей (1кк евро/долларов/стерлингов, в странах с меньшей валютой 5кк), отвечая на вопросы и разводя зрителя на отправку смс. Является излюбленной едой фотожабберов и развлекательных педерач.  В 2019 году в одном из выпусков должны были играть звёзды «Что? Где? Когда?» Друзь и Сиднев. Сыграли они так себе, забрав всего 200 тысяч рублёв.

Здесь два момента, которые могут потенциально сломать правиловое разделение на предложения. Во-первых, вопросительный знак в конце названия передачи «Кто хочет стать миллионером?», который, тем не менее, не заканчивает предложение, а, во-вторых, название передачи «Что? Где? Когда», которое как бы и состоит из трёх коротких предложений, но тем не менее в данном случае является именем собственным в другом предложении.

In [1]:
from razdel import sentenize

In [2]:
text = "«Кто хочет стать миллионером?» — педерача, в которой актеры гонятся за 3кк рублей (1кк евро/долларов/стерлингов, в странах с меньшей валютой 5кк), отвечая на вопросы и разводя зрителя на отправку смс. Является излюбленной едой фотожабберов и развлекательных педерач. В 2019 году в одном из выпусков должны были играть звёзды «Что? Где? Когда?» Друзь и Сиднев. Сыграли они так себе, забрав всего 200 тысяч рублёв."

In [3]:
sents = list(sentenize(text))

In [4]:
print([sent.text for sent in sents])

['«Кто хочет стать миллионером?» — педерача, в которой актеры гонятся за 3кк рублей (1кк евро/долларов/стерлингов, в странах с меньшей валютой 5кк), отвечая на вопросы и разводя зрителя на отправку смс.', 'Является излюбленной едой фотожабберов и развлекательных педерач.', 'В 2019 году в одном из выпусков должны были играть звёзды «Что?', 'Где?', 'Когда?»', 'Друзь и Сиднев.', 'Сыграли они так себе, забрав всего 200 тысяч рублёв.']


Сломалось, но частично. Кейс с «миллионером» обработался корректно — видимо, из-за того, что дальше слова шли с маленькой буквы, а вот «Что? Где? Когда?» разбилось на три предложения, и тот факт, что вокруг стояли кавычки, не помог токенизатору: видим даже, что эти кавычки «прилипли» к соответствующим разделённым предложениям.

> Вообще, razdel очень легко сломать просто тем, что предложение начинается с маленькой буквы, что распространено в сети, или же инициалами, при условии, что между ними стоит пробел (типографически это некорректно, там надо неразрывный ставить, но на практике это постоянно смешивается).

## 2. Токенизация Mystem vs razdel.tokenize (2 балла)


Токенизируйте текст с помощью razdel и с помощью Mystem. Найдите различия в токенизациях. Что по вашему работает лучше на приведенном тексте?

Кажется, что у нас нет доступа к методу токенизации напрямую ни через обёртку, ни через Mystem напрямую, поэтому получим грамматический анализ для всего текста и достанем токены из поля `text`.

Стоит отметить, что Mystem оставляет и пробельные токены (те, которые состоят только из пробела или перехода на новую строку). На самом деле, это удобно, когда мы лемматизируем текст и потом хотим соединить его обратно как есть, но в данном случае просто очень сильно загромождает вывод. Это единственная эвристика на Python, которую мы применим: не будем выводить чисто пробельные токены. Остальное оставляем без изменений.

In [5]:
text = """
Вторым и третьим открытыми белыми карликами стали Сириус B и Процион B. В 1844 году директор Кёнигсбергской обсерватории Фридрих Бессель, анализируя данные наблюдений, которые велись с 1755 года, обнаружил, что Сириус, ярчайшая звезда земного неба, и Процион периодически, хотя и весьма слабо, отклоняются от прямолинейной траектории движения по небесной сфере[5]. Бессель пришёл к выводу, что у каждой из них должен быть близкий спутник. Сообщение было встречено скептически, поскольку слабый спутник оставался ненаблюдаемым, а его масса должна была быть достаточно велика — сравнимой с массой Сириуса и Проциона, соответственно.

В январе 1862 года Элвин Грэхэм Кларк, юстируя 18-дюймовый рефрактор, самый большой на то время телескоп в мире (Dearborn Telescope), впоследствии поставленный семейной фирмой Кларков в обсерваторию Чикагского университета, обнаружил в непосредственной близости от Сириуса тусклую звёздочку. Это был спутник Сириуса, Сириус B, предсказанный Бесселем[6]. А в 1896 году американский астроном Д. М. Шеберле открыл Процион B, подтвердив тем самым и второе предсказание Бесселя.

В 1915 году американский астроном Уолтер Сидней Адамс измерил спектр Сириуса B. Из измерений следовало, что его температура не ниже, чем у Сириуса A (по современным данным, температура поверхности Сириуса B составляет 25 000 K, а Сириуса A — 10 000 К), что, с учётом его в 10 000 раз меньшей, чем у Сириуса A, светимости указывает на очень малый радиус и, соответственно, высокую плотность — 106 г/см3 (плотность Сириуса ~0,25 г/см3, плотность Солнца ~1,4 г/см3).
"""

In [6]:
from razdel import tokenize
from pymystem3 import Mystem


mystem = Mystem()

In [7]:
def tokenize_mystem(text_input, mystem_instance):
    analysis = mystem_instance.analyze(text_input)
    return [item['text']
            for item in analysis if 'text' in item and item['text'].strip()]

In [8]:
def tokenize_razdel(text_input):
    tokens_substrings = list(tokenize(text_input))
    tokens = [token.text for token in tokens_substrings]
    return tokens

In [9]:
%%time

# Токенизируем с Mystem
mystem_tokens = tokenize_mystem(text, mystem)

print(mystem_tokens)

['Вторым', 'и', 'третьим', 'открытыми', 'белыми', 'карликами', 'стали', 'Сириус', 'B', 'и', 'Процион', 'B', '. ', 'В', '1844', 'году', 'директор', 'Кёнигсбергской', 'обсерватории', 'Фридрих', 'Бессель', ', ', 'анализируя', 'данные', 'наблюдений', ', ', 'которые', 'велись', 'с', '1755', 'года', ', ', 'обнаружил', ', ', 'что', 'Сириус', ', ', 'ярчайшая', 'звезда', 'земного', 'неба', ', ', 'и', 'Процион', 'периодически', ', ', 'хотя', 'и', 'весьма', 'слабо', ', ', 'отклоняются', 'от', 'прямолинейной', 'траектории', 'движения', 'по', 'небесной', 'сфере', '[', '5', ']', '. ', 'Бессель', 'пришёл', 'к', 'выводу', ', ', 'что', 'у', 'каждой', 'из', 'них', 'должен', 'быть', 'близкий', 'спутник', '. ', 'Сообщение', 'было', 'встречено', 'скептически', ', ', 'поскольку', 'слабый', 'спутник', 'оставался', 'ненаблюдаемым', ', ', 'а', 'его', 'масса', 'должна', 'была', 'быть', 'достаточно', 'велика', ' — ', 'сравнимой', 'с', 'массой', 'Сириуса', 'и', 'Проциона', ', ', 'соответственно', '.', 'В', 'январ

In [10]:
%%time

# Токенизируем с razdel
razdel_tokens = tokenize_razdel(text)

print(razdel_tokens)

['Вторым', 'и', 'третьим', 'открытыми', 'белыми', 'карликами', 'стали', 'Сириус', 'B', 'и', 'Процион', 'B', '.', 'В', '1844', 'году', 'директор', 'Кёнигсбергской', 'обсерватории', 'Фридрих', 'Бессель', ',', 'анализируя', 'данные', 'наблюдений', ',', 'которые', 'велись', 'с', '1755', 'года', ',', 'обнаружил', ',', 'что', 'Сириус', ',', 'ярчайшая', 'звезда', 'земного', 'неба', ',', 'и', 'Процион', 'периодически', ',', 'хотя', 'и', 'весьма', 'слабо', ',', 'отклоняются', 'от', 'прямолинейной', 'траектории', 'движения', 'по', 'небесной', 'сфере', '[', '5', ']', '.', 'Бессель', 'пришёл', 'к', 'выводу', ',', 'что', 'у', 'каждой', 'из', 'них', 'должен', 'быть', 'близкий', 'спутник', '.', 'Сообщение', 'было', 'встречено', 'скептически', ',', 'поскольку', 'слабый', 'спутник', 'оставался', 'ненаблюдаемым', ',', 'а', 'его', 'масса', 'должна', 'была', 'быть', 'достаточно', 'велика', '—', 'сравнимой', 'с', 'массой', 'Сириуса', 'и', 'Проциона', ',', 'соответственно', '.', 'В', 'январе', '1862', 'года

В большинстве случаев токенизация почти одинаковая, но есть несколько сложных случаев, которые позволяют сказать, что токенизация у razdel нам подходит больше.

Во-первых, слово *18-дюймовый*. Mystem ошибочно разделил его на три токена, тогда как здесь всё-таки вернее считать его одним словом, как полнобуквенную форму "восемнадцатидюймовый" — это прилагательное. Такое разделение, вероятно, и ошибочно повлияет на анализ и лемматизацию.

Во-вторых, Mystem не умеет токенизировать десятичные дроби с запятой (типа 1,4), а razdel — умеет. У razdel десятичная дробь — это один токен, у Mystem их три.

Следующий недостаток, который очень мелкий и на самом деле исправляется простым `strip.()` — то, что Mystem оставляет пробелы вместе со знаками препинания. В прочем, как я уже сказал, в данном случае это скорее не баг, а фича, потому что так потом очень удобно собирать текст обратно. Токенизатор из razdel возвращает «чистые» токены.

Тем не менее, есть один кейс, в котором Mystem сработал лучше, чем razdel: это обозначение кубического сантиметра *см3*. Mystem оставил это как один токен, а razdel отделил тройку от сокращения — здесь первый вариант представляется более правильным.

То есть получается, что Mystem наивно работает с пунктуацией и отделяет её от слова в любом случае, не обращая внимание ни на что, а razdel с этой пунктуацией работает чуть умнее. Однако razdel, возможно, наивнее работает с числами.

Ни один из токенизаторов не справился с тем, чтобы оставить *10 000* и другие большие числа как один токен, но, наверное, это уже не их проблема: по правилам там должен быть не пробел, а какой-нибудь разделитель типа апострофа.

Ну и, разумеется, Mystem токенизировал текст намного медленнее, чем razdel, но это нормально, потому что Mystem — не такинизатор, а анализатор, и токены мы просто достали из анализа как побочный продукт.

## 3. Лемматизация Mystem vs Pymorphy (2 балла)

Лемматизируйте текст с помощью mystem и pymorphy. Найдите различия в лемматизации. Что по вашему работает лучше на приведенном тексте?

Важно: для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат. Анализируйте только значимые различия, а не технические особенности (не сравнивайте скорость работы и удобность интерфейса).

В майстеме убедитесь, что используется дизамбигуация.

In [11]:
from pymorphy3 import MorphAnalyzer


mystem = Mystem(disambiguation=True)
morph = MorphAnalyzer()

In [12]:
def lemmatize_pymorphy(tokens_list, morph_analyzer):
    lemmas = []
    for token in tokens_list:
        p = morph_analyzer.parse(token)[0]
        lemmas.append(p.normal_form)
    return lemmas

In [13]:
%%time

# Лемматизация с Mystem
mystem_lemmas_raw = mystem.lemmatize(text)

mystem_lemmas = [lemma for lemma in mystem_lemmas_raw if lemma.strip()]

print(mystem_lemmas)

['второй', 'и', 'третий', 'открытый', 'белый', 'карлик', 'становиться', 'сириус', 'B', 'и', 'процион', 'B', '. ', 'в', '1844', 'год', 'директор', 'кенигсбергский', 'обсерватория', 'фридрих', 'бессель', ', ', 'анализировать', 'данные', 'наблюдение', ', ', 'который', 'вестись', 'с', '1755', 'год', ', ', 'обнаруживать', ', ', 'что', 'сириус', ', ', 'яркий', 'звезда', 'земной', 'небо', ', ', 'и', 'процион', 'периодически', ', ', 'хотя', 'и', 'весьма', 'слабо', ', ', 'отклоняться', 'от', 'прямолинейный', 'траектория', 'движение', 'по', 'небесный', 'сфера', '[', '5', ']', '. ', 'бессель', 'приходить', 'к', 'вывод', ', ', 'что', 'у', 'каждый', 'из', 'они', 'должный', 'быть', 'близкий', 'спутник', '. ', 'сообщение', 'быть', 'встречать', 'скептически', ', ', 'поскольку', 'слабый', 'спутник', 'оставаться', 'ненаблюдаемый', ', ', 'а', 'его', 'масса', 'должный', 'быть', 'быть', 'достаточно', 'большой', ' — ', 'сравнимый', 'с', 'масса', 'сириус', 'и', 'процион', ', ', 'соответственно', '.', 'в', 'я

In [14]:
%%time

# Лемматизация с Pymorphy3
mystem_tokens = tokenize_mystem(text, mystem)

pymorphy_lemmas = lemmatize_pymorphy(mystem_tokens, morph)

print(pymorphy_lemmas)

['второй', 'и', 'третий', 'открытый', 'белый', 'карлик', 'стать', 'сириус', 'b', 'и', 'процион', 'b', '. ', 'в', '1844', 'год', 'директор', 'кёнигсбергский', 'обсерватория', 'фридрих', 'бессель', ', ', 'анализировать', 'дать', 'наблюдение', ', ', 'который', 'вестись', 'с', '1755', 'год', ', ', 'обнаружить', ', ', 'что', 'сириус', ', ', 'яркий', 'звезда', 'земной', 'небо', ', ', 'и', 'процион', 'периодически', ', ', 'хотя', 'и', 'весьма', 'слабо', ', ', 'отклоняться', 'от', 'прямолинейный', 'траектория', 'движение', 'по', 'небесный', 'сфера', '[', '5', ']', '. ', 'бессель', 'прийти', 'к', 'вывод', ', ', 'что', 'у', 'каждый', 'из', 'они', 'должный', 'быть', 'близкий', 'спутник', '. ', 'сообщение', 'быть', 'встретить', 'скептически', ', ', 'поскольку', 'слабый', 'спутник', 'оставаться', 'ненаблюдаемый', ', ', 'а', 'он', 'масса', 'должный', 'быть', 'быть', 'достаточно', 'большой', ' — ', 'сравнимый', 'с', 'масса', 'сириус', 'и', 'процион', ', ', 'соответственно', '.', 'в', 'январь', '1862'

In [15]:
# Посмотрим только на отличающееся, чтобы сэкономить время
different_lemmas = [(m_lemma, p_lemma) for m_lemma, p_lemma in zip(
    mystem_lemmas, pymorphy_lemmas) if m_lemma != p_lemma]
print(different_lemmas)

[('становиться', 'стать'), ('B', 'b'), ('B', 'b'), ('кенигсбергский', 'кёнигсбергский'), ('данные', 'дать'), ('обнаруживать', 'обнаружить'), ('приходить', 'прийти'), ('встречать', 'встретить'), ('его', 'он'), ('грэхэм', 'грэхэма'), ('Dearborn', 'dearborn'), ('Telescope', 'telescope'), ('поставлять', 'поставить'), ('обнаруживать', 'обнаружить'), ('звездочка', 'звёздочка'), ('B', 'b'), ('предсказывать', 'предсказать'), ('д', 'далее'), ('шеберль', 'шеберл'), ('открывать', 'открыть'), ('B', 'b'), ('то', 'тем'), ('второй', 'второе'), ('измерять', 'измерить'), ('B', 'b'), ('его', 'он'), ('низкий', 'ниже'), ('A', 'a'), ('B', 'b'), ('K', 'k'), ('A', 'a'), ('учет', 'учёт'), ('меньший', 'малый'), ('A', 'a')]


В целом, лемматизация Mystem лучше, потому что есть попытка разрешения неоднозначности, которая иногда работает, что позволило, например, правильно лемматизировать *второе* как *второй* (PyMorphy3 не справился). Хотя со словоформой *данные* не справился никто: Mystem оставил как существительное plurale tantum, а PyMorphy3 проанализировал как глагол *дать* — хотя в данном случае это всё же не причастие, а отглагольное прилагательное, то есть правильным анализом была бы лемма *данный*.

Есть и несколько субъективных критериев, по которым я считаю, что Mystem лучше справился с задачей. Mystem оставляет вид глагола, который был у лемматизируемой словоформы, а Pymorphy3 приводит всё к совершенному виду. С точки зрения лингвистики парадигма Mystem правильная, потому что вид — это словоклассифицирующая, а не словоизменительная категория, и два глагола в видовой паре имеют разные лексемы. Видимо, парадигма авторов PyMorphy и OpenCorpora в данном вопросе отличается. Ещё один такой случай — притяжательные местоимения, где Mystem оставляет их форму (*его*), тогда как Pymorphy3 самым вероятным анализом выбирает *он*. Хотя, скорее всего, это как раз не вопрос разных лингвистических верований, а результат невозможности разрешения омонимии у Pymorphy3, так как форма *его* ещё и форма р.п. и в.п. для личного *он*.

Но есть одна вещь, в которой Pymorphy3 лучше: он сохраняет букву *ё* в леммах, тогда как Mystem либо оставляет, как было в словоформе, либо и вовсе «деёфицирует» все леммы. А ещё очень удивило, что Pymorphy3 единственную букву *д* лемматизировал как *далее*! По всей видимости — результат того самого вероятностного выбора на основе словаря (OpenCorpora), где оно, очевидно, так и размечено. Очень смелое и рискованное предположение, но в данном случае сработавшее идеально!

## 4. Лемматизация в SpaCy (2 балла)

С помощью Spacy (модель для русского языка) лемматизируйте тот же текст. Проверьте есть ли различия с Mystem и Pymoprhy.

Будем использовать большую модельку, потому что а зачем намеренно снижать качество, когда можно лучше?

In [17]:
import spacy

nlp = spacy.load("ru_core_news_lg")

> Я попытался найти способ передавать в SpaCy не текст, а готовый список токенов, чтобы исключить влияние токенизации, однако адекватного и простого не нашёл — а найденные не удовлетворяли моим критериям красоты и качества кода и адекватности затрачиваемых усилий. Поэтому, увы. Stanza с этим справляется намного изящнее, а сейчас мы будем страдать от несоответствия токенов.

In [18]:
%%time
doc = nlp(text)

CPU times: user 74.8 ms, sys: 3.49 ms, total: 78.3 ms
Wall time: 91.2 ms


In [19]:
spacy_lemmas = [
    token.lemma_ for token in doc]

print(spacy_lemmas)

['\n', 'второй', 'и', 'третий', 'открытый', 'белый', 'карлик', 'стать', 'сириус', 'b', 'и', 'процион', 'b.', 'в', '1844', 'год', 'директор', 'кёнигсбергский', 'обсерватория', 'фридрих', 'бессель', ',', 'анализировать', 'данные', 'наблюдение', ',', 'которые', 'вестись', 'с', '1755', 'год', ',', 'обнаружить', ',', 'что', 'сириус', ',', 'ярчайшая', 'звезда', 'земной', 'небо', ',', 'и', 'процион', 'периодически', ',', 'хотя', 'и', 'весьма', 'слабый', ',', 'отклоняться', 'от', 'прямолинейный', 'траектория', 'движение', 'по', 'небесный', 'сфере[5', ']', '.', 'бессель', 'прийти', 'к', 'вывод', ',', 'что', 'у', 'каждый', 'из', 'них', 'должный', 'быть', 'близкий', 'спутник', '.', 'сообщение', 'было', 'встретить', 'скептически', ',', 'поскольку', 'слабый', 'спутник', 'оставаться', 'ненаблюдаемый', ',', 'а', 'он', 'масса', 'должный', 'быть', 'быть', 'достаточно', 'большой', '—', 'сравнимый', 'с', 'масса', 'сириус', 'и', 'проциона', ',', 'соответственно', '.', '\n\n', 'в', 'январь', '1862', 'год',

Ну, в целом, ужасненько...

Прежде всего стоит напомнить, что в SpaCy в качестве лемматизатора для русского языка используется Pymorphy3, в который передаются разборы из обученной модели SpaCy: то есть Pymorphy3 лемматизирует словоформы уже зная их грамматические характеристики, задачу по разрешению неоднозначности берёт на себя SpaCy. Поэтому лемматизации присуще некоторые особенности лемматизации Pymorphy3: склонность переводить глаголы к лексемам несовершенного вида, например.

Однако мы можем наблюдать, что разбор SpaCy часто мешает, а не помогает. Многие слова просто остаются в своей исходной форме: *которые*, *ярчайшая*, *чикагского*, *данным*. Не помог анализ SpaCy и в выявлении притяжательных местоимений.

Ещё есть последствия кривой токенизации: из-за того, что квадратная скобочка от сноски приклеилась к словоформе *сфере*, она правильно не разобралась и не лемматизировалась.

Лемматизация Mystem из этих трёх всё ещё остаётся самой аккуратной, лемматизации SpaCy я бы присвоил второе место (но помним, кто там выполняет всю черновую работу), и, наконец, «чистый» Pymorphy3 всё же справляется хуже всех по объективным причинам, но не намного хуже SpaCy.

> Вообще, самая лучшая лемматизация (и парсинг) для русского языка у моделей из Stanza и UDPipe2 (есть на Синтагрусе и на Тайге и там, и там) — но вы нам его почему-то не дали, а перевыполнять мне лень.

## 5*. LSH (2 балла)

*необязательное задание чтобы получить 10 баллов

Попробуйте искать дубликаты в настоящих текстах. Например, можете взять https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/anna_karenina.txt или https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/besy_dostoevsky.txt (или любой другой корпус)

Используйте код из семинара для нахождения кандидатов в дубликаты (шинглы -> минхэш - lsh) и рассчитайте реальную меру Жаккара между полученными кандидатами. Настройте параметры k, num_hash_functions, bands так чтобы результаты получались адекватные (мера Жаккара хотя бы больше нуля). 

(Можете взять 500-1000 текстов если весь корпус обрабатывается слишком долго)

In [20]:
import hashlib
from collections import defaultdict
from tqdm.notebook import tqdm

### 1. Загружаем данные и собираем тексты

Будем использовать тестовый сет корпуса Синтагрус в UD - просто достанем все предложения как тексты.

In [1]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/refs/heads/master/ru_syntagrus-ud-test.conllu

--2025-10-26 13:58:24--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/refs/heads/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15253211 (15M) [text/plain]
Saving to: ‘ru_syntagrus-ud-test.conllu’

ru_syntagrus-ud-tes 100%[===================>]  14.55M  4.58MB/s    in 3.2s    

2025-10-26 13:58:28 (4.58 MB/s) - ‘ru_syntagrus-ud-test.conllu’ saved [15253211/15253211]



In [21]:
texts = []
file_path = 'ru_syntagrus-ud-test.conllu'

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        if line.startswith('# text ='):
            # Извлекаем текст, убираем префикс пробелы
            texts.append(line.replace('# text =', '').strip())

print(f"Всего извлечено предложений: {len(texts)}")

Всего извлечено предложений: 8800


### 2. Функции и алгоритм

In [22]:
def get_shingles(text, k=5):
    """генерирует список шинглов из строки"""
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return shingles

In [23]:
def hash_string(s):
    """хеширует строку и возвращает число"""
    return int(hashlib.md5(s.encode('utf8')).hexdigest(), 16)

In [24]:
def generate_hash_functions(k):
    """генерирует k хеш-функций"""
    functions = []
    for i in range(k):
        functions.append(lambda x, i=i: hash_string(x + str(i)))
    return functions

In [25]:
def compute_minhash_signature(shingles, hash_funcs):
    """вычисляет minhash-сигнатуру для списка шинглов"""
    signature = []
    for hash_func in hash_funcs:
        # Применяем хэш-функцию ко всем шинглам и берём минимум
        min_hash = min(hash_func(shingle) for shingle in shingles)
        signature.append(min_hash)
    return signature

В функции ниже сделаем одно исправление по сравнению с семинарским кодом. Для хранения бакетов будем использовать не списки, а множества, чтобы одни и те же тексты не попадали в один бакет. Иначе получится, что кандидаты могут содержать пары из одного и того же текста (с одним индексом).

In [26]:
def lsh(signatures, bands):
    """Разрезает сигнатуры на куски (bands) и группирует"""
    buckets = defaultdict(
        set)  # не используем list, чтобы одинаковые тексты не попали в один бакет

    # Проверяем, что число хэшей делится на число бэндов
    if len(signatures[0]) % bands != 0:
        raise ValueError(
            "Number of hash functions must be divisible by the number of bands")

    band_length = len(signatures[0]) // bands

    for idx, sig in enumerate(signatures):
        for b in range(bands):
            start = b * band_length
            end = start + band_length
            band = tuple(sig[start:end])
            buckets[band].add(idx)

    return buckets

In [27]:
def find_similar_texts(texts_list, k, num_hashes, bands):
    """Находит кандидатов в дубликаты используя MinHash и LSH."""
    hash_funcs = generate_hash_functions(num_hashes)
    signatures = []
    shingles_list = []

    for text in tqdm(texts_list):
        shingles = get_shingles(text, k)
        shingles_list.append(shingles)

        # Обрабатываем пустые тексты, если они есть
        if shingles:
            signature = compute_minhash_signature(shingles, hash_funcs)
            signatures.append(signature)
        else:
            # Добавляем "пустую" сигнатуру
            signatures.append([0] * num_hashes)

    buckets = lsh(signatures, bands)

    candidates = set()
    for bucket in buckets.values():
        if len(bucket) > 1:
            unique_indices = sorted(list(bucket))
            # Создаем все уникальные пары (i, j) где i < j
            for i in range(len(unique_indices)):
                for j in range(i + 1, len(unique_indices)):
                    candidates.add((unique_indices[i], unique_indices[j]))

    return candidates, shingles_list

В функции расчёта меры Жаккарда сделаем одну важную проверку: будем возвращать 0, если оба сета шинглов пустые. Иначе будет много ложноположительных результатов на очень коротких предложениях.

In [28]:
def get_jaccard_from_shingles(shingles_x, shingles_y):
    """Считает меру Жаккара для двух готовых множеств шинглов."""

    # Обработка пустых множеств
    if not shingles_x or not shingles_y:
        return 0.0

    intersection = len(shingles_x & shingles_y)
    union = len(shingles_x | shingles_y)

    return intersection / union

### 3. Запускаем: подбираем параметры и определяем лучшее

In [29]:
# Параметры
SHINGLE_K = 5
NUM_HASHES = 100
BANDS = 20

print(f"Запуск LSH на {len(texts)} текстах...")
print(f"Параметры: k={SHINGLE_K}, Хэш-функций={NUM_HASHES}, Бэнды={BANDS}")

Запуск LSH на 8800 текстах...
Параметры: k=5, Хэш-функций=100, Бэнды=20


In [30]:
%%time

candidates, all_shingles = find_similar_texts(
    texts,
    k=SHINGLE_K,
    num_hashes=NUM_HASHES,
    bands=BANDS
)

print(f"Найдено {len(candidates)} пар-кандидатов.")

  0%|          | 0/8800 [00:00<?, ?it/s]

Найдено 86 пар-кандидатов.
CPU times: user 1min 7s, sys: 493 ms, total: 1min 7s
Wall time: 1min 7s


Представляется, что проверять просто больше нуля не очень эффективно, поэтому возьмём порог 0.5, чтобы находить настоящие дубликаты, а не просто что-то отдалённо похожее.

In [31]:
%%time

print("--- Проверка кандидатов (Мера Жаккара > 0.5) ---")

found_duplicates = 0

for i, j in candidates:
    # Получаем меру Жаккара из уже посчитанных шинглов
    shingles_i = all_shingles[i]
    shingles_j = all_shingles[j]
    jaccard_score = get_jaccard_from_shingles(shingles_i, shingles_j)

    # Выводим только если схожесть больше 0.5
    if jaccard_score > 0.5:
        found_duplicates += 1
        print(f"\nПара: ({i}, {j}), Мера Жаккара: {jaccard_score:.4f}")
        print(f"  Текст {i}: {texts[i]}")
        print(f"  Текст {j}: {texts[j]}")

print("Всего найдено дубликатов:", found_duplicates)

--- Проверка кандидатов (Мера Жаккара > 0.5) ---

Пара: (5937, 5943), Мера Жаккара: 1.0000
  Текст 5937: Фокин.
  Текст 5943: Фокин.

Пара: (3405, 4824), Мера Жаккара: 0.8571
  Текст 3405: Ничего подобного.
  Текст 4824: Ничего подобного!

Пара: (2031, 2036), Мера Жаккара: 0.7328
  Текст 2031: На прошлой неделе Министерство образования опубликовало список вузов, допущенных к проведению образовательного эксперимента.
  Текст 2036: А на прошлой неделе Министерство образования опубликовало список из 112 вузов, допущенных к проведению эксперимента.

Пара: (414, 420), Мера Жаккара: 0.5294
  Текст 414: - Иметь-то имеют, - сказала Марья Поликарповна.
  Текст 420: - Да, да, - сказала Марья Поликарповна.

Пара: (474, 8790), Мера Жаккара: 1.0000
  Текст 474: - Пожалуйста.
  Текст 8790: - Пожалуйста.

Пара: (8212, 8360), Мера Жаккара: 0.7091
  Текст 8212: КТО СДАЕТ ПРОДУКТ ВТОРИЧНЫЙ, ТОТ СНАБЖАЕТСЯ ОТЛИЧНО.
  Текст 8360: КТО СДАЕТ ПРОДУКТ ВТОРИЧНЫЙ, ТОТ ПИТАЕТСЯ ОТЛИЧНО.

Пара: (1190, 8019), Мера

В целом, заданные параметры сработали неплохо: мы нашли 86 кандидатов, из них 30 оказались адекватными дубликатами. Считать меру Жаккарда для 86 пар намного лучше, чем для 8800 пар.

Но всё же кажется, что много потенциальных дубликатов мы упускаем, поэтому сделаем алгоритм более чувствительным, заставим его генерировать больше кандидатов. Для этого увеличим band и немного снизим длину шинглов (мы видели, что тут есть много очень коротких предложений). Это должно немного замедлить проверку, но по идеи дать лучший результат.

In [32]:
# Параметры (второй набор)
SHINGLE_K = 4
NUM_HASHES = 100
BANDS = 25

print(f"Запуск LSH на {len(texts)} текстах...")
print(f"Параметры: k={SHINGLE_K}, Хэш-функций={NUM_HASHES}, Бэнды={BANDS}")

Запуск LSH на 8800 текстах...
Параметры: k=4, Хэш-функций=100, Бэнды=25


In [33]:
%%time

candidates, all_shingles = find_similar_texts(
    texts,
    k=SHINGLE_K,
    num_hashes=NUM_HASHES,
    bands=BANDS
)

print(f"Найдено {len(candidates)} пар-кандидатов.")

  0%|          | 0/8800 [00:00<?, ?it/s]

Найдено 328 пар-кандидатов.
CPU times: user 1min 16s, sys: 513 ms, total: 1min 16s
Wall time: 1min 16s


In [34]:
%%time

print("--- Проверка кандидатов (Мера Жаккара > 0.5) ---")

found_duplicates = 0

for i, j in candidates:
    # Получаем меру Жаккара из уже посчитанных шинглов
    shingles_i = all_shingles[i]
    shingles_j = all_shingles[j]
    jaccard_score = get_jaccard_from_shingles(shingles_i, shingles_j)

    # Выводим только если схожесть больше 0.5
    if jaccard_score > 0.5:
        found_duplicates += 1
        print(f"\nПара: ({i}, {j}), Мера Жаккара: {jaccard_score:.4f}")
        print(f"  Текст {i}: {texts[i]}")
        print(f"  Текст {j}: {texts[j]}")

print("Всего найдено дубликатов:", found_duplicates)

--- Проверка кандидатов (Мера Жаккара > 0.5) ---

Пара: (2031, 2036), Мера Жаккара: 0.7656
  Текст 2031: На прошлой неделе Министерство образования опубликовало список вузов, допущенных к проведению образовательного эксперимента.
  Текст 2036: А на прошлой неделе Министерство образования опубликовало список из 112 вузов, допущенных к проведению эксперимента.

Пара: (2728, 2731), Мера Жаккара: 0.8235
  Текст 2728: Личный зачет - 10-8-6-5-4-3-2-1, учитывались результаты всех гонок;
  Текст 2731: Личный зачет - 10-6-4-3-2-1, учитывались результаты всех гонок;

Пара: (6751, 7832), Мера Жаккара: 1.0000
  Текст 6751: - Да.
  Текст 7832: - Да.

Пара: (474, 7969), Мера Жаккара: 0.8182
  Текст 474: - Пожалуйста.
  Текст 7969: - Пожалуйста!

Пара: (6970, 8769), Мера Жаккара: 0.7500
  Текст 6970: - Ладно, - сказал он.
  Текст 8769: - Ладно, - сказал я.

Пара: (6502, 6606), Мера Жаккара: 0.7500
  Текст 6502: Энтузиазм?
  Текст 6606: Энтузиазм!

Пара: (2731, 2732), Мера Жаккара: 0.5875
  Текст 2731

Мы нашли немного больше дубликатов, причём, кажется, немного более релевантных (то есть мера Жаккара в целом выше), но при этом сгенерировали почти в 4 раза больше кандидатов. Видимо, параметры можно подбирать и дальше, но без золотого стандарта — то есть размеченных вручную «истинных» дубликатов — это будет трудно.

### 4. А если всё-таки брутфорс?

Алгоритм, который мы применили выше, спасает нас от брутфорса — сравнения каждого текста с каждым. И всё-таки интересно: если это сделать, сколько это займёт времени и какой будет результат?

In [39]:
bruteforce_duplicates = []
JACCARD_THRESHOLD = 0.5

n = len(texts)

print(f"Запуск полного перебора для {n} текстов...")
print(f"Будет выполнено {n * (n - 1) // 2} сравнений.")

Запуск полного перебора для 8800 текстов...
Будет выполнено 38715600 сравнений.


In [40]:
%%time

for i in tqdm(range(n)):
    for j in range(i + 1, n):
        # Получаем уже готовые множества шинглов
        shingles_i = all_shingles[i]
        shingles_j = all_shingles[j]

        score = get_jaccard_from_shingles(shingles_i, shingles_j)

        if score > JACCARD_THRESHOLD:
            bruteforce_duplicates.append((i, j, score))

  0%|          | 0/8800 [00:00<?, ?it/s]

CPU times: user 5min, sys: 1.37 s, total: 5min 1s
Wall time: 5min


In [41]:
print(
    f"Полный перебор нашел {len(bruteforce_duplicates)} пар с Jaccard > {JACCARD_THRESHOLD}")

Полный перебор нашел 36 пар с Jaccard > 0.5


Что ж, LSH отработал чуть больше минуты (не считая жалкие миллисекунды для проверки кандидатов), а брутфорс работал 5 минут. При этом мы убедились, что очень хорошо настроили наши параметры — с помощью LSH мы нашли 35 пар текстов с мерой Жаккара > 0.5, а брутфорс (который точный) нашёл 36, то есть мы упустили всего одну пару.

При этом надо понимать, что сложность у алгоритма с брутфорсом квадратичная, и, например, если мы удвоим количество текстов, то время выполнения увеличится в 4 раза, а у LSH сложность линейная — так что всё работает и достаточно эффективно!